<a href="https://colab.research.google.com/github/ehsan74814/Preprocessing_Data/blob/main/MultipleLinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 000) .....

In [ ]:
from IPython.display import Image
display(Image("/content/drive/MyDrive/Training ML/simple linear regression/061836706733-web-tete.jpg", width = 500, height = 300))

## About dfset

### **We will study the sales df of one of the largest retailers in the world. Let's figure out what factors influence its revenue. Can factors such as air temperature and fuel cost influence the success of a huge company along with the purchasing power index and seasonal discounts? And how does machine learning minimize costs and increase economic impact?**

**The df contains the following columns:**

**Store: Store number**

**Date: Sales week start date**

**Weekly_Sales: Sales**

**Holiday_Flag: Mark on the presence or absence of a holiday**

**Temperature: Air temperature in the region**

**Fuel_Price: Fuel cost in the region**

**CPI: Consumer price index**

**Unemployment: Unemployment rate**

# 00) Connect the Drive to the Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 0) Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

## Additions

In [ ]:
plt.style.use("fivethirtyeight")
warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:,.0f}".format

# 1) Problem Definition


## what is the goal ? which problem will the algorithm we want to design solve?

# 2) df acquisition

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Training ML/simple linear regression/Walmart_Sales.csv")

# 3) df preprocessing

## Basic df Exploration and df Cleaning

In [ ]:
# Quick review of primary df
df.head(10)

In [ ]:
# df info()
df.info()

In [ ]:
# df shape
df.shape

In [ ]:
# Statistical descriptions
df.describe()

## Handling The missing value

In [ ]:
# checking null
df.isnull().sum()

In [ ]:
# convert columns name to lower case ====> goal ===> Avoid possible errors
df.columns = df.columns.str.lower()

## Handling the duplicated df points

In [ ]:
# checking duplicated
df.duplicated().sum()

## Converting the date column to date and time df type

In [ ]:
# Correcting the df format
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")

In [ ]:
df.info()

In [ ]:
# Creating a column_list
column_list = df.columns.tolist()

## Handling the outliners in the df

In [ ]:
def detect_outlier(field):
    q1 = df[field].quantile(0.25)
    q3 = df[field].quantile(0.75)
    iqr = q3-q1

    threshold = iqr*1.5
    lb = q1-threshold
    ub = q3+threshold

    outliers = [feature for feature in df[field] if feature<lb or feature>ub]
    return outliers


outlier_features = {}
for i in column_list:
    outlier_features[i] = len(detect_outlier(i))*100/len(df)

In [ ]:
outliers = ["Weekly_Sales", "Temperature" , "Unemployment"]

In [ ]:
df['Holiday_Flag'].value_counts()

In [ ]:
# Function to return outlier bounds
def return_bounds(field):
  q1 = df[field].quantile(0.25)
  q3 = df[field].quantile(0.75)
  iqr = q3-q1

  threshold = iqr*1.5
  lb = q1-threshold
  ub = q3+threshold

  return [ub,lb]

# upper_bound, lowerbound = return_bounds("Weekly_Sales")

In [ ]:
for i in outliers:
    lst = return_bounds(i)
    ub = lst[0]
    median = np.median(df[i])
    df[i] = np.where(df[i]>ub,median,df[i])

In [ ]:
for i in outliers:
    lst = return_bounds(i)
    lb = lst[1]
    median = np.median(df[i])
    df[i] = np.where(df[i]<lb,median,df[i])

In [ ]:
outlier_features = {}
for i in column_list:
    outlier_features[i] = len(detect_outlier(i))*100/len(df)

In [ ]:
lst = return_bounds('Unemployment')
lb = lst[1]
ub = lst[0]

In [ ]:
df = df[(df['Unemployment'] < ub) & (df['Unemployment'] > lb)]

## Performing the visualization of the dfset

In [ ]:
df.head()

In [ ]:
df.hist(figsize=(40,20))

## Visualizing the sales over time

In [ ]:
# we have column with the date information
df['Date'] =pd.to_datetime(df['Date'])
df['Date'].head()

In [ ]:
# Create a column month for plotting, yearly df sales

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month

In [ ]:
df.head()

In [ ]:
df['year'].value_counts()

In [ ]:
# sales over months
fig, ax  = plt.subplots(figsize = (20,5))
sns.lineplot(x=df.Date, y = (df.Weekly_Sales/1e6))
plt.xlabel("months")
plt.ylabel("weekly sales")
plt.title("weekly sales Trend", fontdict={"fontsize":16, "color":"red"}, pad = 5)


annot = ax.annotate("", xy=(3,5), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(True)

plt.show()

## Visualizing the holiday effects over sales

In [ ]:
holiday = df[df['Holiday_Flag']==1]
non_holiday = df[df['Holiday_Flag']!=1]

In [ ]:
plt.boxplot(holiday['Weekly_Sales'])

In [ ]:
plt.boxplot(non_holiday['Weekly_Sales'])

## Creating the heatmap to check for the df correlations


In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True, cmap ="YlGnBu")
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12)

## Feature **Engineering**

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [ ]:
# We can remove the date data now because it will not be giving us any important information about the prediction
df.drop(['Date'],axis=1,inplace=True)

# 4) Model selection

# 5) Data spliting

## Feature scaling and data modelling


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Creating the target and the data separation

X = df.drop(['Weekly_Sales'],axis=1)
y = df['Weekly_Sales']

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_data = scaler.fit_transform(X)

In [ ]:
# Creating the test-train split
X_train, X_test, y_train, y_test = train_test_split(scaled_data, y, test_size=0.2, random_state=50)

# 6) Model Training

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
model = LinearRegression().fit(X_train, y_train)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
# Calculating the mean squared error
mse = mean_squared_error(y_test,y_predict)

In [ ]:
# Calculating root mean squared error
rmse = np.sqrt(mse)

In [ ]:
r2_score(y_test,y_predict)

In [ ]:
model.coef_

In [ ]:
model.intercept_

# 7) Model evaluation

# 8) model tunning and optimization

#9) model utilization

#10) Development and maintence